<a href="https://colab.research.google.com/github/suryatorati/DeepfakeDetection_GANS/blob/main/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os

In [ ]:
image_dir_real = '/content/gdrive/MyDrive/training_real'
image_dir_fake = '/content/gdrive/MyDrive/training_fake'

In [ ]:


def preprocess_image(image):
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0
    return image

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = preprocess_image(image)
    return image


real_image_paths = [os.path.join(image_dir_real, filename) for filename in os.listdir(image_dir_real) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]
fake_image_paths = [os.path.join(image_dir_fake, filename) for filename in os.listdir(image_dir_fake) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]


real_images = [load_image(path) for path in real_image_paths]


fake_images = [load_image(path) for path in fake_image_paths]

In [ ]:
def create_discriminator_model(input_shape=(128, 128, 3)):
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", input_shape=input_shape),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

discriminator = create_discriminator_model()
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0002, 0.5), metrics=["accuracy"])


In [ ]:
X_real = np.array(real_images)
X_fake = np.array(fake_images)

y_real = np.ones((X_real.shape[0], 1))
y_fake = np.zeros((X_fake.shape[0], 1))

X_train = np.concatenate((X_real, X_fake), axis=0)
y_train = np.concatenate((y_real, y_fake), axis=0)

discriminator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
127/127 [==============================] - 29s 223ms/step - loss: 0.8332 - accuracy: 0.5634
Epoch 2/10
127/127 [==============================] - 28s 222ms/step - loss: 0.4975 - accuracy: 0.7641
Epoch 3/10
127/127 [==============================] - 30s 235ms/step - loss: 0.3275 - accuracy: 0.8831
Epoch 4/10
127/127 [==============================] - 32s 253ms/step - loss: 0.2094 - accuracy: 0.9529
Epoch 5/10
127/127 [==============================] - 27s 215ms/step - loss: 0.1253 - accuracy: 0.9822
Epoch 6/10
127/127 [==============================] - 30s 236ms/step - loss: 0.0689 - accuracy: 0.9980
Epoch 7/10
127/127 [==============================] - 29s 227ms/step - loss: 0.0470 - accuracy: 0.9990
Epoch 8/10
127/127 [==============================] - 28s 216ms/step - loss: 0.0290 - accuracy: 1.0000
Epoch 9/10
127/127 [==============================] - 30s 235ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 10/10
127/127 [==============================] - 35s 279ms/step - l

In [ ]:
def detect_deepfake_image(image, threshold=0.5):
    image = np.expand_dims(image, axis=0)
    prediction = discriminator.predict(image)

    if prediction > threshold:
        return "This image is likely a real."
    else:
        return "This image appears to be deepfake."

# realImage='/content/gdrive/MyDrive/training_real/real_00142.jpg'
# fakeImage='/content/gdrive/MyDrive/training_fake/easy_187_1110.jpg'

image_to_check = load_image('/content/gdrive/MyDrive/training_fake/mid_236_1111.jpg')
result = detect_deepfake_image(image_to_check)
print(result)

1/1 [==============================] - 0s 48ms/step
This image appears to be deepfake.
